In [2]:
# Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Generate captions for images using default beam search parameters."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import math
import os
from os import listdir, path

import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

import PIL.Image
import PIL.ImageOps
import PIL.ImageFont
import PIL.ImageDraw
import textwrap
from io import BytesIO

from im2txt import configuration
from im2txt import inference_wrapper
from im2txt.inference_utils import caption_generator
from im2txt.inference_utils import vocabulary

In [8]:
# Choose the trained model (2 or 3)
model_number = "3"
model_path = "C:/Users/win8/Documents/PF/im2txt/model/train/model.ckpt-"+model_number+"000000"
vocab_path = "C:/Users/win8/Documents/PF/im2txt/data/mscoco/word_counts"+model_number+".txt"
tf.logging.set_verbosity(tf.logging.INFO)

# Build the inference graph.
g = tf.Graph()
with g.as_default():
    model = inference_wrapper.InferenceWrapper()
    restore_fn = model.build_graph_from_config(configuration.ModelConfig(), model_path)
g.finalize()

# Create the vocabulary.
vocab = vocabulary.Vocabulary(vocab_path) 

sess = tf.Session(graph=g)
# Load the model from checkpoint.
restore_fn(sess)

# Prepare the caption generator. Here we are implicitly using the default
# beam search parameters. See caption_generator.py for a description of the
# available beam search parameters.
generator = caption_generator.CaptionGenerator(model, vocab)

INFO:tensorflow:Building model.
INFO:tensorflow:Initializing vocabulary from file: C:/Users/win8/Documents/PF/im2txt/data/mscoco/word_counts3.txt
INFO:tensorflow:Created vocabulary with 11520 words
INFO:tensorflow:Loading model from checkpoint: C:/Users/win8/Documents/PF/im2txt/model/train/model.ckpt-3000000
INFO:tensorflow:Successfully loaded checkpoint: model.ckpt-3000000


In [9]:
image_path = "./imagenes/"
filenames = listdir(image_path)
font = PIL.ImageFont.truetype("./fonts/Aaargh.ttf", 16)

for file in filenames:
    try:
        print(image_path+file)
        img = PIL.Image.open(image_path+file).convert('RGBA')
        box = PIL.Image.new('RGBA', img.size, (255,255,255,0))
        draw = PIL.ImageDraw.Draw(box)
        image = open(image_path+file,'rb').read() # Read the image as bytes
        captions = generator.beam_search(sess, image)
        for i, caption in enumerate(captions):
            # Ignore begin and end words.
            sentence = [vocab.id_to_word(w) for w in caption.sentence[1:-1]]
            sentence = " ".join(sentence)
            print("  %d) %s (p=%f)" % (i, sentence, math.exp(caption.logprob)))
            if i==0:
                margin = offset = 0
                for line in textwrap.wrap(sentence, width=52):
                    offset_a = offset
                    offset += font.getsize(line)[1]
                    draw.rectangle(((0, offset_a), (img.size[0], offset)), fill=(0,0,0,128))
                    draw.text((margin, offset_a), line, (255,255,255),font=font)
                out = PIL.Image.alpha_composite(img, box)
                out.show()
                out.save("./resultados/model_"+model_number+"-image_"+file)
    except KeyboardInterrupt:
        break

./imagenes/flickr_1.jpg
  0) a man riding skis down a snow covered slope . (p=0.024156)
  1) a person riding skis down a snow covered slope . (p=0.009599)
  2) a person riding skis on a snowy surface (p=0.004260)
./imagenes/flickr_2.jpg
  0) a dog with a frisbee in its mouth (p=0.001494)
  1) a black and white dog with a frisbee in its mouth . (p=0.001423)
  2) a dog with a frisbee in its mouth . (p=0.001368)
./imagenes/flickr_3.jpg
  0) a man and a woman standing next to each other . (p=0.001038)
  1) a man and a woman standing next to each other holding wine glasses . (p=0.000097)
  2) a man and a woman standing next to each other in a room . (p=0.000056)
./imagenes/flickr_4.jpg
  0) a little boy holding a red frisbee in a yard . (p=0.000093)
  1) a young boy holding a red frisbee in a yard . (p=0.000081)
  2) a little boy holding a red frisbee in his hands . (p=0.000046)
./imagenes/flickr_5.jpg
  0) a group of people sitting on top of a sandy beach . (p=0.000223)
  1) a group of peo